In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import cufflinks as cf
import plotly.express as px
import chart_studio.plotly as py
%matplotlib inline

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
cf.go_offline()


def scale_axis(data: pd.DataFrame, scale:dict) -> pd.DataFrame:
    data = data.copy()
    new_list = []
    for key in data["name"].to_list():
        new_list.append(scale[key])
    data["scale"] = new_list
    data["x"] = data["x"] / data["scale"]
    data["y"] = data["y"] / data["scale"]
    data["z"] = data["z"] / data["scale"]
    return data

def frame_args(duration):
    return {
            "frame": {"duration": duration},
            "mode": "next",
            "fromcurrent": True,
            "transition": {"duration": duration, "easing": "linear"},
            }

# def frame_args_hidden(frames, period):
    

In [2]:
planets_data = pd.read_csv("planets_data.csv", index_col=0)
planets_data = planets_data[planets_data.name != "Venus"]
number_of_planets = planets_data.shape[0]
planets_data

,name,mass,color,radius
0,Sun,1.988470e+30,yellow,696340000.0
1,Mercury,3.285000e+23,orange,2439.7
3,Earth,6.046000e+24,blue,6371.0
4,Mars,6.390000e+23,red,3389.5
5,Jupiter,1.898000e+27,orange,69911.0
6,Saturn,5.683000e+26,brown,58232.0
7,Uranus,8.681000e+25,purple,25362.0
8,Neptune,1.024000e+26,cyan,25622.0


In [14]:
import datetime

steps = 300
scale = {"Sun": 1*10e6, "Mercury": 1*10e7, "Venus":1*10e7, "Earth":1*10e7, "Mars":1*10e7,
         "Jupiter":3*10e7, "Saturn":1*10e7, "Uranus":1*10e7, "Neptune": 1*10e7}
scale = {"Sun": 1, "Mercury": 1, "Venus":1, "Earth":1, "Mars":1,
         "Jupiter":1, "Saturn":1, "Uranus":1, "Neptune": 1}

position_history = pd.read_csv("position_history.csv", index_col=0)
position_history["time"] = pd.to_datetime(position_history["time"]).astype("datetime64[D]")
# position_history = scale_axis(position_history, scale=scale)
position_history = position_history[position_history.name != "Venus"]
position_history["radius"] = np.power(position_history["radius"], 1/4) * 10e6
position_history = position_history.sort_values("time")
time_step = ((max(position_history["time"]) - min(position_history["time"])) / steps).days
position_history["time"] = position_history["time"].dt.strftime("%Y:%m:%d")

x_limit = position_history["x"].abs().max()
y_limit = position_history["y"].abs().max()
z_limit = position_history["z"].abs().max()

position_history = position_history.reset_index()
# position_history.head(number_of_planets)




downsampled_position_history = position_history[0:number_of_planets]
for index in range(number_of_planets*time_step, position_history.shape[0], number_of_planets*time_step):
    downsampled_position_history = downsampled_position_history.append(position_history[index:index+number_of_planets])

print(f"Data shape:{downsampled_position_history.shape}")
downsampled_position_history.head(number_of_planets)

Data shape:(2920, 8)


,index,time,x,y,z,name,radius,mass
0,0,2021:05:19,0.000000e+00,0.000000e+00,0.000000e+00,Sun,1.624446e+09,1.988470e+30
1,0,2021:05:19,2.535684e+12,-3.424295e+12,-1.464563e+12,Neptune,1.265183e+08,1.024000e+26
2,0,2021:05:19,-3.744435e+10,2.801968e+10,1.884933e+10,Mercury,7.028038e+07,3.285000e+23
3,0,2021:05:19,2.181137e+12,-1.850526e+12,-8.413384e+11,Uranus,1.261961e+08,8.681000e+25
4,0,2021:05:19,5.584489e+11,4.565727e+11,1.820606e+11,Jupiter,1.626059e+08,1.898000e+27
5,0,2021:05:19,9.221213e+11,9.524896e+11,3.537990e+11,Saturn,1.553425e+08,5.683000e+26
6,0,2021:05:19,1.799662e+11,1.087726e+11,4.502896e+10,Mars,7.630165e+07,6.390000e+23
7,0,2021:05:19,-1.360144e+11,5.375051e+10,2.330735e+10,Earth,8.934122e+07,6.046000e+24


In [ ]:
fig = px.scatter_3d(downsampled_position_history, x="x", y="y", z="z", color="name", size="radius",
                    animation_frame="index", animation_group="name", template="plotly_dark", opacity=0,
                    color_discrete_sequence=["orange", "crimson", "cornflowerblue", "cyan", 
                                             "khaki", "lightcyan", "chocolate", "green"],
                   )
# size="radius",

scene = dict(xaxis = dict(range=[-x_limit, x_limit], autorange=False, visible=False),
             yaxis = dict(range=[-y_limit, y_limit], autorange=False, visible=False),
             zaxis = dict(range=[-z_limit, z_limit], autorange=False, visible=False), 
             )

updatemenus = [
                {
                "buttons": [
                        {
                        "args": [None, frame_args(30)],
                        "label": "&#9654;", # play symbol
                        "method": "animate",
                        },
                    
                        {
                        "args": [[None], frame_args(0)],
                        "label": "&#9724;", # pause symbol
                        "method": "animate",
                        },
                    ],
                },
            ]

marker = dict(opacity=0)

sliders = [
            {
            "steps": [
                {
                    "args": [[frame.name], frame_args(0)],
                    "label": str(index),
                    "method": "animate",
                }
                for index, frame in enumerate(fig.frames)
            ],
            }
        ]
               

fig.update_layout(height=800 ,scene_aspectmode='cube', scene = scene, sliders=sliders, updatemenus=updatemenus)
fig

## Before saving clear simulation (eg. turn cell to markdown) or the file will take 100mb

In [ ]:
# Import data
import time
import numpy as np

from skimage import io

vol = io.imread("https://s3.amazonaws.com/assets.datacamp.com/blog_assets/attention-mri.tif")
volume = vol.T
r, c = volume[0].shape

# Define frames
import plotly.graph_objects as go
nb_frames = 68

fig = go.Figure(frames=[go.Frame(data=go.Surface(
    z=(6.7 - k * 0.1) * np.ones((r, c)),
    surfacecolor=np.flipud(volume[67 - k]),
    cmin=0, cmax=200
    ),
    name=str(k) # you need to name the frame for the animation to behave properly
    )
    for k in range(nb_frames)])

# Add data to be displayed before animation starts
fig.add_trace(go.Surface(
    z=6.7 * np.ones((r, c)),
    surfacecolor=np.flipud(volume[67]),
    colorscale='Gray',
    cmin=0, cmax=200,
    colorbar=dict(thickness=20, ticklen=4)
    ))


def frame_args(duration):
    return {
            "frame": {"duration": duration},
            "mode": "immediate",
            "fromcurrent": True,
            "transition": {"duration": duration, "easing": "linear"},
        }

sliders = [
            {
                "pad": {"b": 10, "t": 60},
                "len": 0.9,
                "x": 0.1,
                "y": 0,
                "steps": [
                    {
                        "args": [[f.name], frame_args(0)],
                        "label": str(k),
                        "method": "animate",
                    }
                    for k, f in enumerate(fig.frames)
                ],
            }
        ]

# Layout
fig.update_layout(
         title='Slices in volumetric data',
         width=600,
         height=600,
         scene=dict(
                    zaxis=dict(range=[-0.1, 6.8], autorange=False),
                    aspectratio=dict(x=1, y=1, z=1),
                    ),
         updatemenus = [
            {
                "buttons": [
                    {
                        "args": [None, frame_args(50)],
                        "label": "&#9654;", # play symbol
                        "method": "animate",
                    },
                    {
                        "args": [[None], frame_args(0)],
                        "label": "&#9724;", # pause symbol
                        "method": "animate",
                    },
                ],
                "direction": "left",
                "pad": {"r": 10, "t": 70},
                "type": "buttons",
                "x": 0.1,
                "y": 0,
            }
         ],
         sliders=sliders
)

fig.show()

In [ ]:
# steps = []
# slider_range = int(downsampled_position_history.shape[0]/8)
# for i in range(slider_range):
#     step = dict(
#         method="update",
#         visible=False,
#     )
#     steps.append(step)

# for step in steps[::3]:
#     step["visible"]=True
    

# sliders = [dict(
#     steps=steps
# )]


# return {
#             "frame": {"duration": duration},
#             "mode": "next",
#             "fromcurrent": True,
#             "transition": {"duration": duration, "easing": "linear"},
#             }